In [1]:
!pip install efficientnet
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 5.2 MB/s eta 0:00:00


In [3]:
# Import necessary libraries
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import efficientnet.tfkeras as efn
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.utils import class_weight
import numpy as np

In [4]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
# Define the path to your zip file in Google Drive (adjust this path as needed)
zip_path = '/content/drive/MyDrive/archive (3).zip'

# Define the extraction directory
extract_dir = '/content/tumor-dataset/'
os.makedirs(extract_dir, exist_ok=True)

# Unzip the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [36]:
# Define paths to training and testing folders
train_dir = os.path.join(extract_dir, '/content/tumor-dataset/Training')
test_dir = os.path.join(extract_dir, '/content/tumor-dataset/Testing')

In [37]:
import os
for class_name in os.listdir(train_dir):
    class_path = os.path.join(train_dir, class_name)
    if os.path.isdir(class_path):
        print(f"{class_name}: {len(os.listdir(class_path))} images")

meningioma_tumor: 822 images
no_tumor: 395 images
glioma_tumor: 826 images
pituitary_tumor: 827 images


In [38]:
# Enhanced data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

In [39]:
# Only rescaling for testing
test_datagen = ImageDataGenerator(rescale=1./255)

In [40]:
# Load and prepare training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # EfficientNetB7 input size
    batch_size=32,           # Smaller batch size for B7's memory demands
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True
)

Found 2870 images belonging to 4 classes.


In [41]:
# Load and prepare testing data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=False
)

Found 394 images belonging to 4 classes.


In [42]:
# Compute class weights to handle imbalance
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weight_dict = dict(enumerate(class_weights))
print("Class weights:", class_weight_dict)

Class weights: {0: 0.8686440677966102, 1: 0.8728710462287105, 2: 1.8164556962025316, 3: 0.8675937122128174}


In [43]:
# Load pre-trained EfficientNetB7 model
base_model = efn.EfficientNetB7(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freeze base model initially
base_model.trainable = False

In [44]:
# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Dropout to prevent overfitting
x = Dense(train_generator.num_classes, activation='softmax')(x)

In [45]:
# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model (initial training)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [46]:
# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-2)

In [47]:
# Initial training with frozen layers
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=15,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size,
    callbacks=[early_stopping, reduce_lr],
    class_weight=class_weight_dict
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 163s 1s/step - accuracy: 0.4656 - loss: 1.3372 - val_accuracy: 0.4792 - val_loss: 1.7267 - learning_rate: 0.0100
Epoch 2/15
 1/89 ━━━━━━━━━━━━━━━━━━━━ 20s 228ms/step - accuracy: 0.6562 - loss: 1.5251

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


89/89 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.6562 - loss: 1.5251 - val_accuracy: 0.4688 - val_loss: 1.7895 - learning_rate: 0.0100
Epoch 3/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 82s 926ms/step - accuracy: 0.5726 - loss: 1.3110 - val_accuracy: 0.4714 - val_loss: 1.9373 - learning_rate: 0.0100
Epoch 4/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.6562 - loss: 0.7544 - val_accuracy: 0.4688 - val_loss: 1.9853 - learning_rate: 0.0100
Epoch 5/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 103s 519ms/step - accuracy: 0.5958 - loss: 1.2396 - val_accuracy: 0.4948 - val_loss: 1.8770 - learning_rate: 0.0100
Epoch 6/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.5938 - loss: 1.0554 - val_accuracy: 0.4844 - val_loss: 1.8759 - learning_rate: 0.0100
Epoch 7/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 82s 926ms/step - accuracy: 0.5984 - loss: 1.2939 - val_accuracy: 0.4297 - val_loss: 2.5591 - learning_rate: 0.0100
Epoch 8/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - accuracy: 0.5000 - loss: 1.3087 - val_accurac

In [48]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 835ms/step - accuracy: 0.3616 - loss: 2.1167
Test Accuracy: 54.06%


In [49]:
# Save the model
model.save('/content/drive/My Drive/efficientnetb7_brain_tumor_model.h5')

In [50]:
# Optional: Confusion matrix for detailed performance
from sklearn.metrics import confusion_matrix
test_generator.reset()
preds = model.predict(test_generator)
pred_labels = np.argmax(preds, axis=1)
true_labels = test_generator.classes
cm = confusion_matrix(true_labels, pred_labels)
print("Confusion Matrix:\n", cm)
print("Class names:", test_generator.class_indices)

13/13 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step
Confusion Matrix:
 [[24 39 15 22]
 [12 47 18 38]
 [ 4 10 76 15]
 [ 0  5  3 66]]
Class names: {'glioma_tumor': 0, 'meningioma_tumor': 1, 'no_tumor': 2, 'pituitary_tumor': 3}
